In [76]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf
from tensorflow.contrib import layers
import numpy as np

env = gym.make("CartPole-v1")


[2017-01-26 00:57:52,720] Making new env: CartPole-v1


In [97]:
n_actions = env.action_space.n
n_states = env.observation_space.shape[0]


model_name = "deep-integrated-policy-gradient-cartpole-v1.model"
model_path = "/models/" + model_name

graph = tf.Graph()
gsess = tf.InteractiveSession(graph=graph)
with graph.as_default():
    with tf.device("cpu:0"):
        s = tf.placeholder(tf.float32, [n_states])
        step = tf.placeholder(tf.int32, [])
        a = tf.placeholder(tf.int32, [])
        r = tf.placeholder(tf.float32, [])
        

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))


        [Ps, V] = Pipe(
            s,
            T.expand_dims(0)
#             .relu_layer(32, scope='relu_layer', **ops)
            .relu_layer(16, scope='relu_layer2', **ops),
            [
                T.softmax_layer(n_actions, scope='softmax_layer_actor', **ops)
                >> T[0]
            ,
                T.linear_layer(1, scope='linear_layer_critic', **ops)
                >> T[0,0]
            ]
        )
        Psa = Ps[a]
        
        ws = tf.trainable_variables()
        
        gradients_actor = tf.gradients(tf.log(Psa), ws)
        gradients_actor = [ g if g is not None else tf.zeros_like(w) for g, w in zip(gradients_actor, ws) ]
        
        gradients_critic = tf.gradients(V, ws)
        gradients_critic = [ g if g is not None else tf.zeros_like(w) for g, w in zip(gradients_critic, ws) ]
        
        dws = [ tf.placeholder(w.dtype, w.get_shape()) for w in ws ]
        update = [ tf.assign_add(w, dw) for w, dw in zip(ws, dws) ]
        

        writer = tf.summary.FileWriter('/logs/' +  model_name)
        saver = tf.train.Saver()

In [167]:
with tf.Session(graph=graph) as sess:
    l = 0.32
    
    G1 = tf.gradients(tf.log(Psa) * r, ws)
    G1 = [ g if g is not None else tf.zeros_like(w) for g, w in zip(G1, ws) ]
    
    G2 = [ g * r if g is not None else None for g in tf.gradients(tf.log(Psa), ws) ]
    G2 = [ g if g is not None else tf.zeros_like(w) for g, w in zip(G2, ws) ]
    
    
    loss = -tf.reduce_sum(tf.log(Psa) * r)
    gdo = tf.train.RMSPropOptimizer(l)
    G3 = gdo.compute_gradients(-loss)
    G3 = [ g if g is not None else tf.zeros_like(w) for g, w in G3 ]
    
    print G3
    
    updateG2 = [ tf.assign_add(w, l * g) for w, g in zip(ws, G2) ]
    updateGDO = gdo.minimize(loss)
    
    sess.run(tf.global_variables_initializer())
    
    print sess.run(G1, feed_dict={s: [0.1, 0.2, 0.3, 0.4], a: 0, r: 10.3})[0][0][0:5]
    print sess.run(G2, feed_dict={s: [0.1, 0.2, 0.3, 0.4], a: 0, r: 10.3})[0][0][0:5]
    print sess.run(G3, feed_dict={s: [0.1, 0.2, 0.3, 0.4], a: 0, r: 10.3})[0][0][0:5]
    
    W0 = sess.run(ws)
    
    assign = lambda values: sess.run([tf.assign(w, _w) for w, _w in zip(ws, values)])
    
    
    
    print "W0  ", W0[0][0][0:5]
    
    sess.run(updateG2, feed_dict={s: [0.1, 0.2, 0.3, 0.4], a: 0, r: 10.3})
    print "WG2 ", sess.run(ws)[0][0][0:5]
    
    assign(W0)
    sess.run(updateGDO, feed_dict={s: [0.1, 0.2, 0.3, 0.4], a: 0, r: 10.3})
    print "WGDO", sess.run(ws)[0][0][0:5]
    
    

[<tf.Tensor 'gradients_175/relu_layer2/MatMul_grad/tuple/control_dependency_1:0' shape=(4, 16) dtype=float32>, <tf.Tensor 'gradients_175/softmax_layer_actor/MatMul_grad/tuple/control_dependency_1:0' shape=(16, 2) dtype=float32>, <tf.Tensor 'zeros_like_141:0' shape=(16, 1) dtype=float32>]
[-0.00139582  0.00012582  0.00139894 -0.00194586 -0.00380595]
[-0.00139582  0.00012582  0.00139894 -0.00194586 -0.00380595]
[-0.00139582  0.00012582  0.00139894 -0.00194586 -0.00380595]
W0   [ 0.00827023  0.0006204   0.00266503  0.00078333  0.00793814]
WG2  [ 0.00782356  0.00066066  0.00311269  0.00016066  0.00672024]
WGDO [ 0.0077994   0.00066284  0.0031369   0.00012697  0.00665436]


In [179]:
import sympy as sp

x, = sp.symbols(['x'])
-sp.diff((x - 4)**2 / 2, x)

.3 * np.log(0.3) + 0.7 * np.log(0.7)

-0.6108643020548935

In [ ]:
    print optim.compute_gradients

In [98]:
def next_action(actions, get_max=False):
    n = actions.shape[0]
    return choice(n, p=actions) if not get_max else np.argmax(actions)

def discount(rewards, y):
    r_accum = 0.0
    for i in reversed(range(len(rewards))):
        r_accum = rewards[i] = rewards[i] + y * r_accum
        
    return rewards

In [106]:
  
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess, model_path)
    
    r_total = 0
    R = 0.0
    lam = 0.6
    y = 0.5
    
    
    
    for t in range(200000):
        k = 2000.0
        lr = 0.005 * k / (k + t)
#         lr = 0.2
        tmp_dws = [ np.zeros(w.get_shape()) for w in ws ]
        
        _s = env.reset()
        I = 1.0
        r_accum = 0.0
        
        buf = []
        rs = []
        
        while True:
            #calculate Ps
            [_ps, _v]= sess.run([Ps, V], feed_dict={s: _s})
            
            #select action
            _a = next_action(_ps)
            
            #take step
            _s1, _r, done, info = env.step(_a)

            r_total += _r
            r_accum += _r
            
#             if done:
#                 _r = r_accum
#             else:
#                 _r = 0.0
            
            buf.append((_s, _a, _s1))
            rs.append(_r)
            
            
            #update state
            _s = _s1

            if done:
                break
                
        
        
        gs = np.asarray(discount(rs, y))
#         print gs
#         raise Exception()
        
        gs = (gs - np.mean(gs)) / np.std(gs)
        
        
        buf = [ (_s, _a, _g, _s1) for (_s, _a, _s1), _g in zip(buf, gs) ]
        #train
        for _s, _a, _g, _s1 in buf: 
            
            [_gradients_actor, _gradients_critic, _v] = sess.run([gradients_actor, gradients_critic, V], {s: _s, a: _a})
            
            if np.isnan(_gradients_actor[0][0,0]):
                print "g:", _g, "v", _v
                print "actor", _gradients_actor, "critic", _gradients_critic
                raise Exception("NAN")
                
            
            
            d = _g
#             print _v, _g, d
            
            tmp_dws = [ tmp_dw + (_dp_dw) for tmp_dw, _dp_dw, _dv_dw in zip(tmp_dws, _gradients_actor, _gradients_critic)  ]
           
        
        feed_dict = { 
            dw: lr * tmp_dw for dw, tmp_dw in zip(dws, tmp_dws)
        }

        sess.run(update, feed_dict=feed_dict)
                
        
        n = 100
        if t % n == 0:
            print(float(r_total) / n)
#             print(t_ward)
            r_total = 0
            saver.save(sess, model_path)
#             print _v, _g, d
        
#             print _gradients_actor


0.39
21.39
22.12
22.62
24.21
24.41
24.19
21.43
23.25
21.34
22.72
23.99


KeyboardInterrupt: 

[(4, 16), (16, 2), (16, 1)]

In [26]:
_s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    saver.restore(sess, model_path)
    
    for i in range(1000):
        
        _ps = sess.run(Ps, feed_dict={s: _s})
        _a = next_action(_ps, 0)
        
        
        print _s
        
        print [_psa for _psa in _ps], _a
        
        _s, _r, done, info = env.step(_a)
        env.render()
        
        print _s
        
        print("")

        if done:
            print(_r)
            break

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


[ 0.03754155  0.02952458 -0.03543921  0.03825249]
[nan, nan] 0


NoSuchDisplayException: Cannot connect to "None"